# test pyspark install

In [5]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Now you can use PySpark in this notebook
df = spark.createDataFrame([{"hello":"world"}])
df.show()

+-----+
|hello|
+-----+
|world|
+-----+



# stop the spark session

In [2]:
spark.stop()

# start spark with options

In [4]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Now you can use PySpark in this notebook
df = spark.createDataFrame([{"hello":"world"}])
df.show()

+-----+
|hello|
+-----+
|world|
+-----+

